# Endopoint 1

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
df_steam_games = pq.read_table('steam_games.parquet').to_pandas()


In [ ]:
df_steam_games.dropna(subset=['id'], inplace=True)
df_steam_games.dropna(subset=['developer'], inplace=True)

In [ ]:
df_steam_games.tail(5)

In [ ]:
type(df_steam_games['id'][50000])

In [ ]:
df_steam_games[df_steam_games['developer'] == 'none'].shape, df_steam_games.shape

In [ ]:
df = df_steam_games.groupby('id').first().reset_index()

In [ ]:
df.head()

In [ ]:
df['price'] = df['price'].astype(float)

In [ ]:
#SE elimina las columnas que no necesita consultar la consulta
df.drop(columns=['genres','app_name','tags','specs'], axis=1, inplace=True)

In [ ]:
df.to_parquet('df_endpoint1.parquet')

In [ ]:
cantidad = df[df['developer'] == desarrollador].groupby('anio').count()
cantidad_free = df[(df['developer'] == desarrollador) & (df['price'] == 0.0)].groupby('anio').count()

In [ ]:
porc_cont_free = {}
for i in range(len(anio)):
  if anio[i] not in anio_free:
    #print(anio[i],'0%')
    porc_cont_free[anio[i]] = [cantidad_juegos[i],"0%"]
  else:
    for j in range(len(anio_free)):
      if anio_free[j]==anio[i]:
        #print(anio[i], cantidad_juegos_free[j]/cantidad_juegos[i])
        porc_cont_free[anio[i]] = [cantidad_juegos[i], cantidad_juegos_free[j]/cantidad_juegos[i]*100]



# Endopoint 2

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
from unidecode import unidecode

In [ ]:
df_reviews = pq.read_table('data/df_reviews_sentimiento.parquet').to_pandas()
df_reviews.head()

In [ ]:
df_reviews.shape


In [ ]:
type(df_endpoint2['sentiment_analysis'][0])

In [ ]:
df_endpoint2 = df_reviews[['item_id','sentiment_analysis']]

In [ ]:
df_endpoint2 = df_endpoint2[(df_endpoint2['sentiment_analysis'] == 0) | (df_endpoint2['sentiment_analysis'] == 2)]

In [ ]:
df_endpoint2['sentiment_analysis'].value_counts()

In [ ]:
df_steam_games = pq.read_table('data/df_endpoint1.parquet').to_pandas()

In [ ]:
df_steam_games.drop(columns=['price','anio'], axis=1, inplace=True)

In [ ]:
df_steam_games.head()

In [ ]:
df_endpoint2.head()

In [ ]:
resultado = df_steam_games.merge(df_endpoint2, left_on='id', right_on='item_id', how='inner')

In [ ]:
resultado.head()

resultado.shape

In [ ]:
resultado['developer'] = resultado['developer'].str.replace('   ',' ').str.replace('.','')

In [ ]:

resultado['developer'] = resultado['developer'].apply(unidecode)

In [ ]:
words_to_remove = [
    ' incferal interactive macferal interactive linux',
    ' mac amp linux',
    ' interactive mac',
    ' interactive macferal interactive linux',
    ' gamesaspyr macvirtual programming linux',
    ' softwareaspyr mac amp linux',
    ' interactive mac/linux',
    'incferal interactive macferal interactive linux '
]

def clean_developer_name(name):
    for word in words_to_remove:
        name = name.replace(word, '').strip()
    return name

In [ ]:
#Limpieza de caracteres especiales
# Reemplaza '-' por ' ' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace('-', ' ') if x.dtype == "object" else x)

# Reemplaza '!' por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace('!', '') if x.dtype == "object" else x)

# Reemplaza '¡' por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace('¡', '') if x.dtype == "object" else x)

# Reemplaza '?' por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace('?', '') if x.dtype == "object" else x)

# Reemplaza '¿' por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace('¿', '') if x.dtype == "object" else x)

# Reemplaza '"' por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace('"', '') if x.dtype == "object" else x)

# Reemplaza ''' por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace("'", '') if x.dtype == "object" else x)

# Reemplaza ',' por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace(",", '') if x.dtype == "object" else x)

# Reemplaza '&' por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace("&", '') if x.dtype == "object" else x)

# Reemplaza "''" por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace("''", '') if x.dtype == "object" else x)

# Reemplaza "_" por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace("_", '') if x.dtype == "object" else x)

# Reemplaza "_" por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace("[", '') if x.dtype == "object" else x)

# Reemplaza "_" por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace("]", '') if x.dtype == "object" else x)

# Reemplaza "_" por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace(";", '') if x.dtype == "object" else x)

# Reemplaza "_" por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace("(", '') if x.dtype == "object" else x)

# Reemplaza "_" por '' en todas las columnas
resultado = resultado.apply(lambda x: x.str.replace(")", '') if x.dtype == "object" else x)

In [ ]:
resultado['cleaned_developer_column'] = resultado['developer'].apply(clean_developer_name)

In [ ]:
resultado['developer'] = resultado['cleaned_developer_column']

In [ ]:
resultado.drop('item_id',axis=1, inplace=True)

In [ ]:
resultado.head()

In [ ]:
resultado.to_parquet('df_endpoint2.parquet')

# Endopoint 3

In [1]:
import pandas as pd
import pyarrow.parquet as pq

In [ ]:
df_reviews = pq.read_table('df_reviews_sentimiento.parquet').to_pandas()
df_steam_games = pq.read_table('df_steam_games.parquet').to_pandas()


In [ ]:
df_reviews = df_reviews[['user_id','recommend','item_id']]
df_steam_games = df_steam_games[['price','id']]

In [ ]:
type(df_reviews['item_id'][0]),type(df_steam_games['id'][0])

In [ ]:
resultado = df_reviews.merge(df_steam_games, left_on='item_id', right_on='id', how='inner')

In [ ]:
resultado.drop('id',axis=1,inplace=True)
resultado.shape

In [ ]:
df = resultado.drop_duplicates(subset=['user_id', 'item_id'])


In [ ]:
df.head()

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.drop(columns=['level_0','index'], axis=1, inplace=True)

In [ ]:
import pyarrow.parquet as pq
df.to_parquet('df_endpoint3.parquet')

# Endopoint 4

In [ ]:
import pandas as pd
import pyarrow.parquet as pq

In [ ]:
df_steam_games = pq.read_table('data/df_steam_games.parquet').to_pandas()

df_items = pq.read_table('df_user_items.parquet').to_pandas()

df_steam_games = df_steam_games[['genres','id','anio']]
df_items = df_items[['user_id','item_id','playtime_forever','playtime_2weeks']]


In [ ]:

df_items[( df_items['playtime_2weeks'] != 0.0 ) & ( df_items['playtime_forever'] == 0.0)]

df_items.at[253453, 'playtime_forever'] = df_items.at[253453,'playtime_2weeks']
df_items.at[1093823, 'playtime_forever'] = df_items.at[1093823,'playtime_2weeks']
df_items.at[2763031, 'playtime_forever'] = df_items.at[2763031,'playtime_2weeks']

df_items.drop('playtime_2weeks',axis=1, inplace=True)

df = df_items.drop_duplicates()

df.reset_index(inplace=True, drop=True)

df1 = df_steam_games.groupby('id').first().reset_index()

df1 = df1[df1['anio']!=0]

resultado = df1.merge(df, left_on='id', right_on='item_id', how='inner')

In [ ]:
#Se muestra el datafreame
resultado.head()

In [ ]:
#Se guarda el dataframe
resultado.to_parquet('df_endpoint4')

## Endpoint 5

In [ ]:
df_steam_games = pq.read_table('data/df_steam_games_limpio.parquet').to_pandas()
df_reviews = pq.read_table('/data/df_reviews_sentimiento.parquet').to_pandas()

In [ ]:
df_steam_games = df_steam_games[['id','developer','anio']]
df_reviews = df_reviews[['item_id','recommend','sentiment_analysis']]

In [ ]:
df = df_steam_games.groupby('id').first().reset_index()

In [ ]:
df = df[df['anio']!=0]

In [ ]:
df.head()

In [ ]:
df_reviews.head(5)

In [ ]:
resultado = df.merge(df_reviews, left_on='id', right_on='item_id', how='inner')

In [ ]:
resultado.drop('id', axis=1, inplace=True)

In [ ]:
resultado = resultado[resultado['recommend'] == 'true']

In [ ]:
resultado = resultado[resultado['sentiment_analysis'] == 2]

In [ ]:
resultado.head()

In [ ]:
resultado.to_parquet('df_endpoint5.parquet')

In [ ]:
resultado[(resultado['anio'] == 2000) & (resultado['developer'] == 'valve')].shape

In [ ]:
cantidad

In [ ]:
cantidad.reset_index(inplace=True)

In [ ]:
cantidad = cantidad.sort_values(by='anio', ascending=False)

In [ ]:
cantidad = cantidad[['developer','anio']]

In [ ]:
cantidad.head()

In [ ]:
cantidad.set_index('developer', inplace=True)

In [ ]:
cantidad.columns = ['Cantidad']

In [ ]:
primeros_tres = cantidad.iloc[:3]

In [ ]:
primeros_tres.to_dict()